# Scrapeo de datos

In [1]:
import requests # para api y scrapy
import pandas as pd
from bs4 import BeautifulSoup # para scrapy
from tqdm.notebook import tqdm # barra de progreso (porque tarda mucho)
import time # para hacer sleep en medio de scrapeo y evitar nos bloqueen
import re # para expresiones regulares
import base64 # convertir a base64
import random
import unicodedata

#1 Ingestion de Datos API

Se realizo ingestion de datos por medio de API de un geoserver de la alcaldia de La Paz

In [ ]:
# Parámetros base
base_url = "https://sitservicios.lapaz.bo/geoserver/sit/ows"
params = {
    "service": "WFS",
    "version": "2.0.0",
    "request": "GetFeature",
    "typeNames": "sit:tramitesterritoriales",
    "count": 5000,
    "outputFormat": "application/json",
    "srsName": "EPSG:4326"
}

# Número total de features (puedes ajustar si cambia)
total_features = 13061
features_per_page = params["count"]
paginas = range(0, total_features, features_per_page)

# Lista para guardar los registros
todos_los_registros = []

# Descarga paginada
for start_index in tqdm(paginas, desc="Descargando páginas"):
    params["startIndex"] = start_index
    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        for feature in data["features"]:
            props = feature["properties"]
            geometry = feature.get("geometry")

            # Validar que la geometría exista
            if geometry and "coordinates" in geometry:
                lon, lat = geometry["coordinates"]
                props["latitude"] = lat
                props["longitude"] = lon
            else:
                props["latitude"] = None
                props["longitude"] = None

            todos_los_registros.append(props)
    else:
        print(f"⚠️ Error en startIndex={start_index}: código {response.status_code}")

# Crear DataFrame
df = pd.DataFrame(todos_los_registros)

# Guardar como CSV
df.to_csv("tramites_lapaz_api.csv", index=False, encoding="utf-8")

# Mostrar primeras filas
df.head()

Descargando páginas:   0%|          | 0/3 [00:00<?, ?it/s]

,idPCTramite,descripcion,idTipoTramite,fechaRegistro,Solicitante,arquitectoNombre,arquitectoRegistroNacionalCAB,nroInmueble,idProyectoDesarrollo,idTipoObra,...,codigoCatastral,fechaAprobacion,macroDistrito,distritoMunicipal,cantidadPisos,superficieLegal,nombreEdificio,SuperficieConstruida,latitude,longitude
0,1,PERMISO DE CONSTRUCCION,1,2017-11-14T17:04:40.303Z,SALVADOR MAURICIO REVOLLO ALARCON,EDSON SANJINEZ RAMOS,5047,60159,8.0,2.0,...,017009100040000,2017-12-12T20:22:07.723Z,V - SUR,21,4.0,NaN,VIVIENDAS COPLAT,NaN,-16.520795,-68.108548
1,2,PERMISO DE CONSTRUCCION,1,2017-11-14T20:28:34.223Z,MARIA LEONOR APAZA CHOQUE,PAOLA VERÓNICA PHILCO PARDO,11088,115082,8.0,2.0,...,044036000120000,2017-12-05T15:53:13.480Z,V - SUR,18,4.0,288.00,None,NaN,-16.526083,-68.074901
2,3,PERMISO DE CONSTRUCCION,1,2017-11-15T20:16:57.103Z,INES IBONI LANZA DE QUISBERT,JAIME LEONARDO MANTILLA LOPEZ,11922,99179,8.0,2.0,...,006013400070000,2017-12-01T13:23:26.043Z,III - PERIFERICA,13,4.0,219.06,None,NaN,-16.478085,-68.117633
3,4,PERMISO DE CONSTRUCCION,1,2017-11-16T15:06:48.070Z,ELIZABETH DEL CARMEN PEREZ GUZMAN,JULIO CONDE CONDE,8176,130368,8.0,2.0,...,020051900250000,2017-11-29T19:36:00.990Z,IV - SAN ANTONIO,16,5.0,212.99,SOLIZ PEREZ,NaN,-16.486180,-68.097181
4,5,PERMISO DE CONSTRUCCION,1,2017-11-17T13:32:03.923Z,JAVIER VASQUEZ MAMANI,GUILLERMO VLADIMIR MUÑOZ MARQUEZ,4146,135402,8.0,2.0,...,044280500180000,2017-12-15T20:18:32.837Z,V - SUR,18,4.0,480.00,None,NaN,-16.509784,-68.045165


In [ ]:
df.shape

(13923, 28)

In [ ]:
df.describe()

,idPCTramite,idTipoTramite,idProyectoDesarrollo,idTipoObra,NumeroTramite,idInsDocumento,distritoMunicipal,cantidadPisos,superficieLegal,SuperficieConstruida,latitude,longitude
count,13923.00000,13923.000000,10540.000000,10540.000000,13923.000000,1738.000000,13923.000000,13922.000000,13181.000000,3678.000000,13899.000000,13899.000000
mean,14324.36235,3.672772,8.195920,1.958254,33866.794082,197918.293441,11.630970,4.407269,319.944808,1116.961485,-16.504799,-68.116217
std,11778.69327,4.717879,0.887891,0.200017,19895.729710,77785.070623,6.181844,12.780765,1317.029193,2165.885100,0.022056,0.030325
min,1.00000,1.000000,1.000000,1.000000,1.000000,85617.000000,1.000000,0.000000,0.000000,1.000000,-16.597460,-68.170706
25%,3708.50000,1.000000,8.000000,2.000000,22679.000000,119412.750000,6.000000,3.000000,160.000000,374.777500,-16.518600,-68.141072
50%,10440.00000,1.000000,8.000000,2.000000,33109.000000,200862.500000,12.000000,4.000000,215.000000,565.185000,-16.501811,-68.120834
75%,25114.50000,1.000000,9.000000,2.000000,50589.500000,268807.250000,18.000000,5.000000,306.000000,959.842500,-16.489944,-68.098331
max,36615.00000,12.000000,12.000000,2.000000,72331.000000,330875.000000,21.000000,1476.000000,60000.000000,51714.600000,-16.440893,-68.031340


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13923 entries, 0 to 13922
Data columns (total 28 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   idPCTramite                    13923 non-null  int64  
 1   descripcion                    13923 non-null  object 
 2   idTipoTramite                  13923 non-null  int64  
 3   fechaRegistro                  13923 non-null  object 
 4   Solicitante                    13923 non-null  object 
 5   arquitectoNombre               13923 non-null  object 
 6   arquitectoRegistroNacionalCAB  13923 non-null  object 
 7   nroInmueble                    13921 non-null  object 
 8   idProyectoDesarrollo           10540 non-null  float64
 9   idTipoObra                     10540 non-null  float64
 10  TipoProyecto                   10540 non-null  object 
 11  TipoObra                       10540 non-null  object 
 12  NumeroTramite                  13923 non-null 

# 2 Ingestion por medio de Scrapy HTML

Teniendo los IDs se convierte a base64 de cada fila hacemos scrapy para obtener mas infomracion

Ejemplo de Dato

https://sitservicios.lapaz.bo/situtiles/pc/?MDQ0MTA0OTAwMDcwMDAwfDYxMTUzfDIwMjU=

In [ ]:
# Filtrar solo los aprobados que son los que tienen data para scrapper
df = pd.read_csv("tramites_lapaz_api_identificador.csv",dtype={"codigo_catastral": str})
df_filtrado = df[df['resultado']=="APROBADO"]
df_filtrado.to_csv("tramites_aprobados.csv", index=False, encoding="utf-8")

In [ ]:
duplicados = df[df.duplicated("codigoCatastral", keep=False)]
print(duplicados)

KeyError: Index(['codigoCatastral'], dtype='object')

In [ ]:
# api con identificador
df = pd.read_csv("tramites_lapaz_api.csv",dtype={"codigoCatastral": str})

In [ ]:
print(df.shape)
df.info()

(13907, 28)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13907 entries, 0 to 13906
Data columns (total 28 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   idPCTramite                    13907 non-null  int64  
 1   descripcion                    13907 non-null  object 
 2   idTipoTramite                  13907 non-null  int64  
 3   fechaRegistro                  13907 non-null  object 
 4   Solicitante                    13907 non-null  object 
 5   arquitectoNombre               13907 non-null  object 
 6   arquitectoRegistroNacionalCAB  13907 non-null  int64  
 7   nroInmueble                    13905 non-null  object 
 8   idProyectoDesarrollo           10540 non-null  float64
 9   idTipoObra                     10540 non-null  float64
 10  TipoProyecto                   10540 non-null  object 
 11  TipoObra                       10540 non-null  object 
 12  NumeroTramite                  139

In [ ]:
def parse_table_below_header(soup, header_text):
    header = soup.find("h5", string=re.compile(header_text, re.IGNORECASE))
    table_data = []
    if header:
        table = header.find_next("table")
        if table:
            for row in table.find_all("tr"):
                cols = row.find_all("td")
                cols_text = [col.get_text(strip=True) for col in cols]
                table_data.append(cols_text)
    return table_data

# Función para convertir lista de listas a diccionario plano
def flatten_table_data(table_data, prefix=""):
    flat_data = {}
    for row in table_data:
        if len(row) == 2:
            key = normalize_key(row[0])
        elif len(row) > 2:
            key = "_sub_".join(row[:-1])  # separador "sub"
            key = normalize_key(key)
        else:
          continue  # fila vacía o inválida

        # Agregar prefijo si no está vacío
        if prefix:
            key = f"{prefix}_{key}"

        flat_data[key] = row[-1]
    return flat_data

# Función para convertir un valor a base64
def to_base64(valor):
    return base64.b64encode(str(valor).encode("utf-8")).decode("utf-8")

# Función para convertir a snake_case
def to_snake(name):
    name = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    name = re.sub('([a-z0-9])([A-Z])', r'\1_\2', name)
    return name.replace("__", "_").lower()

# Recolectamos los datos
data = []

def normalize_key(text):
    # 1. Pasar a minúscula
    text = text.lower()

    # 2. Reemplazar ñ -> ni
    text = text.replace("ñ", "ni")

    # 3. Quitar tildes usando unicodedata
    text = "".join(
        c for c in unicodedata.normalize("NFD", text)
        if unicodedata.category(c) != "Mn"
    )

    # 4. Reemplazar múltiples espacios por uno
    text = re.sub(r"\s+", " ", text)

    # 5. Reemplazar espacios por "_"
    text = text.replace(" ", "_")

    # 6. Quitar caracteres no deseados (solo letras, números y "_")
    text = re.sub(r"[^a-z0-9_]", "", text)

    return text


In [ ]:
df = pd.read_csv("tramites_aprobados.csv",dtype={"codigoCatastral": str})

In [ ]:
# Activar tqdm en Pandas
tqdm.pandas()

# Convertir a datetime
df["fechaRegistro"] = pd.to_datetime(df["fechaRegistro"], errors="coerce")

# Extraer el año, pero convierte los que no tienen fecha a NaN
df["anio_registro"] = df["fechaRegistro"].dt.year
# convertir a Integer
df["anio_registro"] = df["anio_registro"].astype("Int64")  # entero nullable

df["identificador"] = (
    df["codigoCatastral"].astype(str) + "|" +
    df["NumeroTramite"].astype(str) + "|" +
    df["anio_registro"].astype(str)
)

# convertimos a base64 codigoCatastral
df["identificador_b64"] = df["identificador"].apply(to_base64)

In [ ]:
# Renombrar todas las columnas
df.columns = [to_snake(col) for col in df.columns]
df.to_csv("tramites_lapaz_api_identificador.csv", index=False, encoding="utf-8")
df.head()

,id_pc_tramite,descripcion,id_tipo_tramite,fecha_registro,solicitante,arquitecto_nombre,arquitecto_registro_nacional_cab,nro_inmueble,id_proyecto_desarrollo,id_tipo_obra,...,distrito_municipal,cantidad_pisos,superficie_legal,nombre_edificio,superficie_construida,latitude,longitude,anio_registro,identificador,identificador_b64
0,1,PERMISO DE CONSTRUCCION,1,2017-11-14 17:04:40.303000+00:00,SALVADOR MAURICIO REVOLLO ALARCON,EDSON SANJINEZ RAMOS,5047,60159,8.0,2.0,...,21,4.0,NaN,VIVIENDAS COPLAT,NaN,-16.520795,-68.108548,2017,017009100040000|1|2017,MDE3MDA5MTAwMDQwMDAwfDF8MjAxNw==
1,2,PERMISO DE CONSTRUCCION,1,2017-11-14 20:28:34.223000+00:00,MARIA LEONOR APAZA CHOQUE,PAOLA VERÓNICA PHILCO PARDO,11088,115082,8.0,2.0,...,18,4.0,288.00,NaN,NaN,-16.526083,-68.074901,2017,044036000120000|2|2017,MDQ0MDM2MDAwMTIwMDAwfDJ8MjAxNw==
2,3,PERMISO DE CONSTRUCCION,1,2017-11-15 20:16:57.103000+00:00,INES IBONI LANZA DE QUISBERT,JAIME LEONARDO MANTILLA LOPEZ,11922,99179,8.0,2.0,...,13,4.0,219.06,NaN,NaN,-16.478085,-68.117633,2017,006013400070000|3|2017,MDA2MDEzNDAwMDcwMDAwfDN8MjAxNw==
3,4,PERMISO DE CONSTRUCCION,1,2017-11-16 15:06:48.070000+00:00,ELIZABETH DEL CARMEN PEREZ GUZMAN,JULIO CONDE CONDE,8176,130368,8.0,2.0,...,16,5.0,212.99,SOLIZ PEREZ,NaN,-16.486180,-68.097181,2017,020051900250000|4|2017,MDIwMDUxOTAwMjUwMDAwfDR8MjAxNw==
4,5,PERMISO DE CONSTRUCCION,1,2017-11-17 13:32:03.923000+00:00,JAVIER VASQUEZ MAMANI,GUILLERMO VLADIMIR MUÑOZ MARQUEZ,4146,135402,8.0,2.0,...,18,4.0,480.00,NaN,NaN,-16.509784,-68.045165,2017,044280500180000|5|2017,MDQ0MjgwNTAwMTgwMDAwfDV8MjAxNw==


In [ ]:
df_filter = df[df["codigoCatastral"]=="044104900070000"]
print(df_filter.head(10))
df_filter.to_csv("draft.csv", index=False, encoding="utf-8")

      idPCTramite                            descripcion  idTipoTramite  \
3061        33609  PERMISO DE CONSTRUCCION-LICENCIA AGIL             12   

                        fechaRegistro                    Solicitante  \
3061 2025-03-07 20:58:21.667000+00:00  MARIA VIRGINIA YAMPASI ZABALA   

                   arquitectoNombre  arquitectoRegistroNacionalCAB  \
3061  CHRISTIAN SERGIO YUJRA HUANCA                          18122   

     nroInmueble  idProyectoDesarrollo  idTipoObra  ... distritoMunicipal  \
3061      195973                   NaN         NaN  ...                18   

     cantidadPisos  superficieLegal nombreEdificio SuperficieConstruida  \
3061           3.0            516.8            NaN               354.04   

       latitude  longitude anio_registro               identificador  \
3061 -16.500009 -68.083611          2025  044104900070000|61153|2025   

                         identificador_b64  
3061  MDQ0MTA0OTAwMDcwMDAwfDYxMTUzfDIwMjU=  

[1 rows x 31 columns]

In [ ]:

def consultar_api(fila):
  url = f"https://sitservicios.lapaz.bo/situtiles/pc/?{ fila['identificador_b64']}"
  #print(url)
  try:
    response = requests.get(url,timeout=10)
    soup = BeautifulSoup(response.content, "html.parser")

    # Detectar error en HTML
    # ejemplo https://sitservicios.lapaz.bo/situtiles/pc/?MDQ0MTUwMzAwMDIwMDAwfDExNnwyMDE3
    if "No fue posible procesar la solicitud" in soup.get_text():
        return pd.Series({
            "url": url,
            "error": "rechazada"
        })

    # Pausa random para evitar bloqueos
    #time.sleep(random.uniform(0.3, 1.2))

    # Extraer secciones
    datos_generales_raw = parse_table_below_header(soup, "Datos Generales")
    parametros_raw = parse_table_below_header(soup, "Parámetros de Construcción")
    otros_datos_raw = parse_table_below_header(soup, "Otros datos")

    # Convertir a formato plano
    datos_generales = flatten_table_data(datos_generales_raw,'gral_')
    parametros = flatten_table_data(parametros_raw,'par_')
    otros_datos = flatten_table_data(otros_datos_raw,'otro_')

    # Combinar todos los datos
    row = {
        "url": url,
        "identificador_b64":fila['identificador_b64'],
    }
    # Unimos todas las claves
    row.update(datos_generales)
    row.update(parametros)
    row.update(otros_datos)

    return pd.Series(row)
  except Exception as e:
    return pd.Series({"url": url, "error": str(e)})

In [ ]:
df = pd.read_csv("tramites_aprobados.csv",dtype={"codigo_catastral": str})
df.head(10)

,id_pc_tramite,descripcion,id_tipo_tramite,fecha_registro,solicitante,arquitecto_nombre,arquitecto_registro_nacional_cab,nro_inmueble,id_proyecto_desarrollo,id_tipo_obra,...,distrito_municipal,cantidad_pisos,superficie_legal,nombre_edificio,superficie_construida,latitude,longitude,anio_registro,identificador,identificador_b64
0,9,PERMISO DE CONSTRUCCION,1,2017-11-20 17:18:52.080000+00:00,IRENE MARCELA GONZALES DE CRUZ,ARIEL HUANCA RODRIGUEZ,2033,97760,8.0,2.0,...,18,5.0,365.64,NaN,1232.69,-16.540992,-68.059075,2017.0,044018700330000|9|2017,MDQ0MDE4NzAwMzMwMDAwfDl8MjAxNw==
1,10,PERMISO DE CONSTRUCCION,1,2017-11-21 13:59:08.027000+00:00,ROSARIO MORALES VISCARRA,MARIO GENARO VEIZAGA GUTIERREZ,2626,146441,8.0,2.0,...,18,3.0,300.00,NaN,285.28,-16.511015,-68.088538,2017.0,044287500010000|10|2017,MDQ0Mjg3NTAwMDEwMDAwfDEwfDIwMTc=
2,29,PERMISO DE CONSTRUCCION,1,2017-11-27 14:07:38.037000+00:00,HOTELERA ZEGARRA S.R.L.,NICOLAS BOHORQUEZ PEÑARANDA,11899,287597,5.0,2.0,...,18,5.0,600.00,NaN,2690.27,-16.535837,-68.078996,2017.0,044041500140000|29|2017,MDQ0MDQxNTAwMTQwMDAwfDI5fDIwMTc=
3,37,PERMISO DE CONSTRUCCION,1,2017-11-29 15:02:57.730000+00:00,YESENIA EDOMILIA VELASCO MEJIA,VICTOR RAMOS MAMANI,2567,155392,9.0,2.0,...,6,5.0,215.00,EDO,907.43,-16.501538,-68.141229,2017.0,026000300900000|37|2017,MDI2MDAwMzAwOTAwMDAwfDM3fDIwMTc=
4,56,PERMISO DE CONSTRUCCION,1,2017-12-01 15:47:55.363000+00:00,JOHNY EFRAIN CHOQUE LAURA,VICTOR RAMOS MAMANI,2567,69845,8.0,2.0,...,17,4.0,267.00,VIVIENDA UNIFAMILIAR ANGELICA,531.12,-16.514355,-68.108029,2017.0,034071700090000|56|2017,MDM0MDcxNzAwMDkwMDAwfDU2fDIwMTc=
5,66,PERMISO DE CONSTRUCCION,1,2017-12-04 17:10:15.630000+00:00,ELIZABETH DEL CARMEN PEREZ GUZMAN,JULIO CONDE CONDE,8176,130368,8.0,2.0,...,16,5.0,212.99,NaN,670.81,-16.486180,-68.097181,2017.0,020051900250000|66|2017,MDIwMDUxOTAwMjUwMDAwfDY2fDIwMTc=
6,73,PERMISO DE CONSTRUCCION,1,2017-12-05 14:14:25.137000+00:00,SIMON TAPIA ZAMBRANA,EDUARDO JESUS SOLARES PEREDO,2167,66063,8.0,1.0,...,18,3.0,1142.90,NaN,2394.89,-16.543249,-68.085157,2017.0,044004000160000|73|2017,MDQ0MDA0MDAwMTYwMDAwfDczfDIwMTc=
7,97,PERMISO DE CONSTRUCCION,1,2017-12-08 16:21:58.663000+00:00,PABLO ROBERTO VARGAS ARANDA,ROBERTO PABLO VARGAS HERMOSA,9183,15744,1.0,2.0,...,18,7.0,578.94,NaN,2904.66,-16.540798,-68.077081,2017.0,044000100030000|97|2017,MDQ0MDAwMTAwMDMwMDAwfDk3fDIwMTc=
8,111,PERMISO DE CONSTRUCCION,1,2017-12-11 19:43:23.163000+00:00,ROBERTO CAUNA CORI,OSVALDO QUISPE FERNANDEZ,13123,67480,8.0,2.0,...,11,3.0,160.00,CAUNA YUPANQUI,375.51,-16.479049,-68.137551,2017.0,009008700130000|111|2017,MDA5MDA4NzAwMTMwMDAwfDExMXwyMDE3
9,116,PERMISO DE CONSTRUCCION,1,2017-12-12 17:37:44.350000+00:00,CARLOS JOSE VASQUEZ GRANDCHANT,CARLOS ALCIDES ADRIAZOLA GUILLEN,417,202658,8.0,2.0,...,18,3.0,650.00,NaN,637.66,-16.510176,-68.052958,2017.0,044150300020000|116|2017,MDQ0MTUwMzAwMDIwMDAwfDExNnwyMDE3


In [ ]:
df_resultado = df.progress_apply(consultar_api, axis=1)
df_resultado.to_csv("tramites_html.csv", index=False, encoding="utf-8")

  0%|          | 0/3222 [00:00<?, ?it/s]

In [ ]:
df_resultado.to_csv("tramites_html_identificador.csv", index=False, encoding="utf-8")

# 3. Merge CSVs

In [ ]:
df_api = pd.read_csv("tramites_lapaz_api_identificador.csv",dtype={"codigo_catastral": str})
df_html = pd.read_csv("tramites_html_identificador.csv",dtype={"gral__codigo_catastral": str})

# Normalizar nombres de las llaves
df_html = df_html.rename(columns={"gral__codigo_catastral": "codigo_catastral"})

# Outer join por código catastral
df_merge = pd.merge(
    df_api, df_html,
    on="codigo_catastral",
    how="outer",
    suffixes=("_api", "_html"),
    indicator=True
)

# Crear columna 'fuente'
df_merge["fuente"] = df_merge["_merge"].map({
    "left_only": "API",
    "right_only": "HTML",
    "both": "Ambos"
})
df_merge.drop(columns=["_merge"], inplace=True)

print("Distribución por fuente:")
print(df_merge["fuente"].value_counts())

df_merge.head()

   id_pc_tramite descripcion  id_tipo_tramite fecha_registro solicitante  \
0            NaN         NaN              NaN            NaN         NaN   
1            NaN         NaN              NaN            NaN         NaN   
2            NaN         NaN              NaN            NaN         NaN   
3            NaN         NaN              NaN            NaN         NaN   
4            NaN         NaN              NaN            NaN         NaN   

  arquitecto_nombre  arquitecto_registro_nacional_cab nro_inmueble  \
0               NaN                               NaN          NaN   
1               NaN                               NaN          NaN   
2               NaN                               NaN          NaN   
3               NaN                               NaN          NaN   
4               NaN                               NaN          NaN   

   id_proyecto_desarrollo  id_tipo_obra  ... par__lateral_2 par__mezzanine  \
0                     NaN           NaN  ...

In [ ]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17129 entries, 0 to 17128
Data columns (total 57 columns):
 #   Column                                                               Non-Null Count  Dtype  
---  ------                                                               --------------  -----  
 0   idPCTramite                                                          13907 non-null  float64
 1   descripcion                                                          13907 non-null  object 
 2   idTipoTramite                                                        13907 non-null  float64
 3   fechaRegistro                                                        13907 non-null  object 
 4   Solicitante                                                          13907 non-null  object 
 5   arquitectoNombre                                                     13907 non-null  object 
 6   arquitectoRegistroNacionalCAB                                        13907 non-null  float64
 7   nroI

In [ ]:
df_merged.shape

(17129, 61)

In [ ]:
df_merged.to_csv("tramites_merge_identificador.csv", index=False, encoding="utf-8")